<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_Fine_tuning_masked_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ajustant un model de llenguatge emmascarat**

---
Per a moltes aplicacions de PNL que involucren models Transformer, només podeu agafar un model preentrenat del Hugging Face Hub i ajustar-lo directament a les vostres dades per a la tasca que teniu a l'abast. Sempre que el corpus utilitzat per a la formació prèvia no sigui massa diferent del corpus utilitzat per a l'ajustament, l'aprenentatge per transferència normalment produirà bons resultats.

**Què és l’adaptació de domini?**

L’adaptació de domini (domain adaptation) és el procés pel qual ajustes un model de llenguatge preentrenat a un nou domini (tema o tipus de text) abans de fer-lo servir per a una tasca concreta de Processament del Llenguatge Natural (PNL), com classificació, anàlisi de sentiments, extracció d'entitats, etc.

**Per què cal fer adaptació de domini?**

Els models com BERT, RoBERTa, etc., han estat preentrenats amb textos molt generals (Wikipedia, llibres, Reddit...). Això funciona bé per a moltes aplicacions.

Però si tens dades especialitzades, com per exemple:

* Contractes legals

* Articles científics

* Textos mèdics

* Converses de servei tècnic

Aleshores aquestes dades tenen vocabulari i estil molt específics que el model pot no conèixer o entendre malament. Això pot reduir el rendiment quan l’ajustes a una tasca específica.

**Com funciona el procés?**

El procés es pot resumir en dues etapes:

1. Adaptació de domini (ajust del model de llenguatge)
Agafes el model preentrenat i continues entrenant-lo (fine-tuning) amb textos del teu domini (per exemple, 100.000 articles científics). Aquí no cal que tinguis etiquetes; només fas modelatge de llenguatge (el model aprèn a predir paraules, com feia al preentrenament).

**Objectiu**: que el model absorbeixi millor la terminologia i l'estil del teu corpus.

2. Entrenament per a la tasca específica
Un cop el model entén millor el llenguatge del teu domini, li afegeixes un “cap” (head) per a la tasca concreta (per exemple, classificació binària) i el tornes a ajustar.

Ara el model ja sap “llegir” millor els teus textos → més bons resultats en la tasca.


Utilitzarem un model anomenat DistilBERT que es pot entrenar molt més ràpidament amb poca o cap pèrdua de rendiment en tasques específiques . Aquest model es va entrenar mitjançant una tècnica especial anomenada destil·lació del coneixement, on s'utilitza un gran "model de professor" com BERT per guiar la formació d'un "model d'estudiant" que té molts menys paràmetres.

Amb uns 67 milions de paràmetres, DistilBERT és aproximadament dues vegades més petit que el model base de BERT, la qual cosa es tradueix aproximadament en una acceleració doble en l'entrenament, bé! Vegem ara quins tipus de fitxes preveu aquest model que són les finalitzacions més probables d'una petita mostra de text:

In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
model.summary()

In [3]:
text = "This is a great [MASK]."

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Amb un tokenizer i un model, ara podem passar el nostre exemple de text al model, extreure els logits i imprimir els 5 millors candidats:

1. Fa una passada pel model (com BERT) amb els inputs. Obté les sortides (logits), que són les prediccions per a cada token i cada paraula del vocabulari.

1. Troba la posició del token [MASK] dins dels input_ids. Aquest serà el punt on el model ha de fer la predicció.
1. Agafa els logits corresponents al token [MASK]. Aquest vector té la mida del vocabulari (ex: 30.000 valors), un per a cada paraula possible.
1. Ordena els logits de major a menor (per això el -). Agafa els 5 tokens amb la probabilitat més alta. Són les 5 paraules que el model creu més probables per substituir [MASK].

In [ ]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

# **Dataset**

---

Per mostrar l'adaptació del domini, utilitzarem el famós conjunt de dades de revisió de pel·lícules grans (o IMDb per abreujar-lo), que és un corpus de ressenyes de pel·lícules que s'utilitza sovint per comparar models d'anàlisi de sentiments. En ajustar DistilBERT en aquest corpus, esperem que el model lingüístic adapti el seu vocabulari a partir de les dades de fet de la Viquipèdia en què va ser entrenat prèviament als elements més subjectius de les ressenyes de pel·lícules.

In [ ]:
!pip install datasets

from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

In [ ]:
# Encadenarem les funcions Dataset.shuffle() i Dataset.select() per crear una mostra aleatòria:

sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")

# **Preprocessament de les dades**

---
Preparar el corpus de text de manera eficient per entrenar un model de llenguatge, evitant perdre informació i aprofitant millor el context entre frases o exemples.

**Concepte clau: Concatenar tot el corpus**

Abans de dividir-lo, en lloc de tractar cada frase o exemple per separat (com es fa sovint en classificació, traducció, etc.), aquí es concatena tot el text del corpus en una sola seqüència llarga de tokens i després es divideix en trossos de longitud fixa (per exemple, 512 tokens).

**Per què es fa això?**

1. Si tokenizeges exemple per exemple amb truncation=True:
Exemples massa llargs s'escurcen → perds informació valuosa. No es pot aprofitar context entre frases o exemples consecutius.

1. En canvi, si concatens tot i després parteixes en blocs:
Mantens el màxim context possible. El model veu més relacions i dependències globals.

Ideal per a models que preveuen tokens següents o completacions, com GPT o BERT en entrenament.


In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

La funció, tokenize, rep un lot d’exemples (examples), i en cada exemple agafa el text a la columna "text", el tokenitza amb el tokenizer predefinit (com un BERT tokenizer, per exemple).

Si el tokenizer és de tipus “fast” (basat en Tokenizers de Hugging Face), pot retornar IDs de paraula. Això serveix per fer whole word masking (màscara de paraules senceres) més endavant.
Es guarda word_ids per a cada seqüència tokenitzada.

Retorna el diccionari amb:

* "input_ids" → els tokens codificats com a números

* "attention_mask" → opcionalment, les màscares

* "word_ids" → si aplica

Aquesta línia fa el mapeig del dataset:

* imdb_dataset és un Dataset de Hugging Face amb columnes com text i label.

* .map(...) aplica la funció tokenize_function a cada lot de dades (batched=True).

* Elimina les columnes "text" i "label" del dataset processat, perquè ja no es necessiten per entrenar un model de llenguatge (només necessitem els tokens).

In [ ]:
tokenizer.model_max_length

In [10]:
chunk_size = 128

Per mostrar com funciona la concatenació, agafem unes quantes ressenyes del nostre conjunt de formació amb testimoni i imprimim el nombre de fitxes per revisió:

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

In [ ]:
# A continuació, podem concatenar tots aquests exemples amb una comprensió senzilla de diccionari, de la següent manera:

concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

La longitud total es comprova, així que ara dividim les ressenyes concatenades en trossos de la mida donada per chunk_size. Per fer-ho, iterem sobre les característiques de concatenat_examples i utilitzem una comprensió de llista per crear parts de cada característica. El resultat és un diccionari de fragments per a cada característica:

In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

Com podeu veure en aquest exemple, l'últim tros generalment serà més petit que la mida màxima del tros. Hi ha dues estratègies principals per fer front a això:

Deixeu anar l'últim tros si és més petit que chunk_size.
Relleu l'últim tros fins que la seva longitud sigui igual a chunk_size.
Prenem el primer enfocament aquí, així que englobem tota la lògica anterior en una única funció que podem aplicar als nostres conjunts de dades tokenitzats:

In [14]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

Tingueu en compte que a l'últim pas de group_texts() creem una nova columna d'etiquetes que és una còpia de la d'input_ids. Com veurem aviat, això es deu al fet que en el modelatge de llenguatge emmascarat l'objectiu és predir fitxes emmascarades aleatòriament al lot d'entrada i, mitjançant la creació d'una columna d'etiquetes, proporcionem la veritat bàsica del nostre model de llenguatge per aprendre.

Ara apliquem group_texts() als nostres conjunts de dades tokenitzats mitjançant la nostra funció de confiança Dataset.map():

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

Com s'esperava de la nostra funció group_texts() anterior, sembla idèntic als input_ids descodificats, però llavors, com pot el nostre model aprendre alguna cosa? Ens falta un pas clau: inserir fitxes [MASK] en posicions aleatòries a les entrades! Vegem com ho podem fer sobre la marxa durant l'ajustament amb un col·lector de dades especial.

# **Ajustant DistilBERT amb l'API Trainer**

---
Ajustar un model de llenguatge emmascarat és gairebé idèntic a afinar un model de classificació de seqüències, com vam fer al capítol 3. L'única diferència és que necessitem un col·lector de dades especial que pugui emmascarar aleatòriament alguns dels testimonis de cada lot de textos. Afortunadament, Transformers ve preparat amb un DataCollatorForLanguageModeling dedicat només per a aquesta tasca. Només hem de passar-li el tokenizer i un argument mlm_probability que especifiqui quina fracció de fitxes cal emmascarar. Escollirem un 15%, que és la quantitat que s'utilitza per a BERT i una opció habitual a la literatura.


In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Per veure com funciona l'emmascarament aleatori, donem uns quants exemples al col·lector de dades. Com que espera una llista de dictats, on cada dict representa un sol fragment de text contigu, primer iterem sobre el conjunt de dades abans d'alimentar el lot al col·lador. Suprimim la clau "word_ids" d'aquest col·lector de dades perquè no ho espera:

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

Quan s'entrenen models per al modelatge de llenguatge emmascarat, una tècnica que es pot utilitzar és emmascarar paraules senceres juntes, no només fitxes individuals. Aquest enfocament s'anomena emmascarament de paraules senceres. Si volem utilitzar l'emmascarament de paraules senceres, haurem de construir nosaltres mateixos un recopilador de dades. Un recopilador de dades és només una funció que pren una llista de mostres i les converteix en un lot, així que fem-ho ara! Utilitzarem els identificadors de paraules calculats anteriorment per fer un mapa entre els índexs de paraules i les fitxes corresponents, després decidirem aleatòriament quines paraules emmascarar i aplicar aquesta màscara a les entrades. Tingueu en compte que les etiquetes són totes -100 excepte les corresponents a les paraules de màscara.

In [19]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

Ara que tenim dos recopiladors de dades, la resta de passos d'ajustament són estàndard. L'entrenament pot trigar una estona a Google Colab si no tens la sort d'aconseguir una GPU P100 mítica, de manera que primer reduirem la mida del conjunt d'entrenament a uns quants milers d'exemples. No us preocupeu, encara tindrem un model d'idioma força decent! Una manera ràpida de rebaixar un conjunt de dades a Conjunts de dades és mitjançant la funció Dataset.train_test_split()

In [ ]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install datasets
from datasets import load_dataset


In [28]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, TFDistilBertForMaskedLM


In [29]:


tf_train_dataset = model.prepare_tf_dataset(
    downsampled_dataset["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    downsampled_dataset["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

A continuació, configurem els nostres hiperparàmetres d'entrenament i compilem el nostre model. Utilitzem la funció create_optimizer() de la biblioteca Transformers, que ens ofereix un optimitzador AdamW amb una disminució de la taxa d'aprenentatge lineal. També fem servir la pèrdua integrada del model, que és la predeterminada quan no s'especifica cap pèrdua com a argument per compilar(), i establim la precisió d'entrenament a "mixed_float16". Tingueu en compte que si feu servir una GPU Colab o una altra GPU que no tingui suport accelerat de float16, probablement hauríeu de comentar aquesta línia.

A més, vam configurar un PushToHubCallback que desarà el model al Hub després de cada època. Podeu especificar el nom del repositori al qual voleu enviar amb l'argument hub_model_id (en particular, haureu d'utilitzar aquest argument per enviar-lo a una organització). Per exemple, per impulsar el model a l'organització huggingface-course, hem afegit hub_model_id="huggingface-course/distilbert-finetuned-imdb". De manera predeterminada, el repositori utilitzat estarà al vostre espai de noms i s'anomenarà després del directori de sortida que heu definit, de manera que en el nostre cas serà "lewtun/distilbert-finetuned-imdb".

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-imdb", tokenizer=tokenizer
)

***Perplexity per LLM***

---
**Cross-Entropy (Entropia creuada)**

**Què és?**
Mesura com “dolenta” és la predicció del model en relació amb la resposta correcta.

**Com funciona?**

Compara la probabilitat que el model dona a la paraula correcta amb el que hauria de predir. Si la paraula correcta té probabilitat baixa, la cross-entropy és alta.

**Valor ideal:**

Com més petit, millor (0 seria perfecte, però inassolible).

**Unitat:**

Sovint es mesura en nats (si es fa servir logaritme natural) o bits (si es fa servir log base 2).

**Perplexity (Perplexitat)**

**Què és?**

És una transformació de la cross-entropy. Més intuïtivament, indica “quantes opcions possibles considera el model” per predir la següent paraula.

**Com funciona?**

És el valor de exp(cross-entropy).
Si la cross-entropy és 1, la perplexity serà e^1 ≈ 2.72.

**Valor ideal:**

Com més baix, millor. Una perplexitat de 1 vol dir que el model està 100% segur de cada paraula. Una perplexitat de 100 vol dir que el model està força perdut.



**Relació entre les dues:**

Perplexity = exp(cross-entropy)

Totes dues mesuren la mateixa cosa: com de bones són les prediccions del model, però la perplexity és més fàcil d'interpretar en el context de llenguatge natural.



In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])

In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

# **Utilitzant el nostre model ajustat**

---



In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="huggingface-course/distilbert-base-uncased-finetuned-imdb"
)

In [ ]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

**El nostre model ha adaptat clarament els seus pesos per predir paraules que estan més fortament associades a les pel·lícules!**